In [7]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import yfinance as yf #stock api


In [46]:
#Tickers
start = '2000-09-02'
end = '2021-07-02'

ticker_adjclose = yf.download('MSFT, CAT, AAPL, WYNN, FCEL, SPRT', period = "ytd", start = start, group_by = 'column',auto_adjust=True, threads = True)['Close']

[*********************100%***********************]  6 of 6 completed


In [47]:
#Given data
exp_ret = 0.02
log_returns = np.log(1+ticker_adjclose.pct_change().dropna())
cov = log_returns.cov()
weight = np.full((len(cov),1), 1/len(cov)) #Start from equal weight portfolio
bounds = ((0,1),)*len(cov)

In [48]:
#Function and constrains
def risk_func(weight):
    return np.dot(weight.T, np.dot(cov,weight))

def Sum_to_one(weight):
    return np.sum(weight)-1

def Min_return(weight):
    return exp_ret - np.sum(log_returns.mean() * weight)

constraints = ({'type': 'eq', 'fun': Sum_to_one},{'type': 'eq', 'fun': Min_return})

In [51]:
#optimizer
optimizer = minimize(risk_func, weight, method = 'SLSQP', bounds = bounds, constraints = constraints)
weight_optim = optimizer.x

In [52]:
#check constraints just in case
[risk_func(weight_optim), Sum_to_one(weight_optim), Min_return(weight_optim)]

[0.0004528583120371921, 2.2746249328520207e-12, 0.01864503633158654]